In [60]:
import numpy as np
import matplotlib.pyplot as plt

In [61]:
# take in vector of unknowns, and build A matrix and c-vector
def from_x_to_matrix(x):
    n = int((-3 + np.sqrt(9+8*x.size))//2)
    k = x.size - n
    A = np.zeros((n, n))
    c = np.zeros(n)
    
    #Insert first k coefficients into matrix
    end = 0
    for j in range(n):
        start = end
        end = start + n-j
        A[j, j:] = x[start:end]
        A[j+1:,j] = A[j, j+1:]
    c = x[-n:] #Insert last n coefficients into vector
    return A, c

In [69]:
# Calculate g for single point z
def gi(x, zi):
    A, c = from_x_to_matrix(x)
    return (zi-c).dot(A.dot(zi-c)) - 1

# Calculate residual r for single point z
def r(x, zi, wi):
    return np.maximum(wi * gi(x, zi), 0)

# Calculate residual vector
def R(x, Z, W):
    m, n = Z.shape
    R = np.zeros(m)
    for i in range(R.size):
        R[i] = r(x, Z[i], W[i])
    return R

# Calculate objective function
def f(x, Z, W):
    m, n = Z.shape
    return np.sum(R(x, Z, W)**2)

# Calculate gradient of g for single point z
def dgi(x, zi):
    n = zi.size
    dg = np.zeros(x.size)
    end = 0
    A, c = from_x_to_matrix(x)
    v = zi - c
    for j in range(n):
        start = end
        end = start + n - j
        dg[start] = v[j]**2
        dg[start+1:end] = 2 * v[j] * v[start+1:end]
    dg[-n:] = -2 * A.dot(v)
    return dg

# Calculate gradient of residual r for single point z
# h is the gi-value for the given point
def dri(x, zi, wi, g = None):
    n = zi.size
    dr = np.zeros(x.size)
    if g == None:
        g = gi(x, zi)
    return (g > 0) * dgi(x, zi) * wi

# Calculate jacobian of residual vector R
def jacobi(x, Z, W, g = None):
    m, n = Z.shape
    J = np.zeros((m, x.size))
    for i in range(m):
        J[i] = dri(x, Z[i], W[i], g)
    return J

# Calculate gradient of objective function
def df(x, Z, W, g = None):
    return 2 * (jacobi(x, Z, W, g).T).dot(R(x, Z, W))

In [70]:
x = np.array((1, 0, 0.25, 2, 0))
# Z = np.array((0, 0, 0, 1, 0, 2, 0, 3, 0, 4)).reshape(5, 2)
# Z = np.array((0,0, 1, 0, 2, 0, 3, 0, 4, 0)).reshape(5, 2)
# Z = np.array((0, 0, 1, 1, 2, 2, 3, 3, 4, 4)).reshape(5,2)
Z = np.array((2, 0, 2, 1, 2, 2, 2, 3, 2, 4)).reshape(5, 2)
W = np.array((1, 1, 1, 1, 1)) 
print('Z: \n', Z)
print('W: \n', W)

Z: 
 [[2 0]
 [2 1]
 [2 2]
 [2 3]
 [2 4]]
W: 
 [1 1 1 1 1]


In [71]:
gi(x, Z[2])

0.0

In [72]:
r(x, Z[2], W[2])

0.0

In [73]:
R(x, Z, W)

array([ 0.  ,  0.  ,  0.  ,  1.25,  3.  ])

In [74]:
f(x, Z, W)

10.5625

In [75]:
df(x, Z, W)

array([   0.  ,    0.  ,  118.5 ,    0.  ,  -15.75])

In [148]:
m, n = Z.shape
k = n*(n+1)//2

x = np.random.randn(n+k)
p = np.random.randn(n+k)
p = p/np.linalg.norm(p)
f0 = f(x, Z, W)
g = df(x, Z, W).dot(p)
if g == 0:
    print("p: \n", p)
    print(df(x, Z, W))
else:
    print("g = %e" %g)
    for ep in 10.0**np.arange(2, -9, -1):
        g_app = (f(x+ep*p, Z, W)-f0)/ep
        error = abs(g_app-g)/abs(g)
        print('ep = %e, error = %e, g_app = %e' % (ep,error, g_app))

g = -1.730711e+02
ep = 1.000000e+02, error = 9.914474e-01, g_app = -1.480211e+00
ep = 1.000000e+01, error = 9.144739e-01, g_app = -1.480211e+01
ep = 1.000000e+00, error = 3.830155e-01, g_app = -1.067822e+02
ep = 1.000000e-01, error = 5.242077e-02, g_app = -1.639986e+02
ep = 1.000000e-02, error = 5.426424e-03, g_app = -1.721320e+02
ep = 1.000000e-03, error = 5.445045e-04, g_app = -1.729769e+02
ep = 1.000000e-04, error = 5.446913e-05, g_app = -1.730617e+02
ep = 1.000000e-05, error = 5.447093e-06, g_app = -1.730702e+02
ep = 1.000000e-06, error = 5.447048e-07, g_app = -1.730711e+02
ep = 1.000000e-07, error = 5.385178e-08, g_app = -1.730711e+02
ep = 1.000000e-08, error = 1.347834e-08, g_app = -1.730712e+02
